# 实验二：空间拓扑关系运算和生成

## 数据类型定义

In [1]:
import math
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

# 定义结点类
class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f"({self.x}, {self.y})"
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash((self.x, self.y))

# 定义弧段类
class Edge:
    def __init__(self, start_node, end_node):
        self.start = start_node
        self.end = end_node
    
    def __repr__(self):
        return f"{self.start} -> {self.end}"

## 输入数据

In [2]:
# 结点列表
nodes = [
    Node(0, 0), Node(2, 0), Node(2, 2), Node(0, 2), 
    Node(1, 0), Node(1, 1), Node(2, 1), 
    Node(0.5, 0.5), Node(1.5, 0.5), Node(1.5, 1.5), Node(0.5, 1.5), 
    Node(2, 3), Node(3, 3), Node(3, 2)
]

# 边列表
edges = [
    Edge(nodes, nodes), Edge(nodes, nodes), Edge(nodes, nodes), Edge(nodes, nodes), 
    Edge(nodes, nodes), Edge(nodes, nodes), Edge(nodes, nodes), Edge(nodes, nodes), 
    Edge(nodes, nodes), Edge(nodes, nodes), Edge(nodes, nodes), Edge(nodes, nodes), 
    Edge(nodes, nodes), Edge(nodes, nodes), Edge(nodes, nodes), Edge(nodes, nodes)
]

print(f"结点总数: {len(nodes)}")
print(f"边总数: {len(edges)}")

结点总数: 14
边总数: 16


## 算法实现/生成

In [ ]:
# 判断多边形的方向，返回面积
def polygon_area(polygon):
    """
    计算多边形的有向面积
    输入polygon为多边形的结点坐标列表，首尾结点相同
    返回值>0表示逆时针，<0表示顺时针
    """
    area = 0
    n = len(polygon)
    for i in range(n - 1):  # 这里用 n - 1，因为第一个点和最后一个点相同
        area += polygon[i].x * polygon[i + 1].y
        area -= polygon[i].y * polygon[i + 1].x
    return area / 2  # 不取绝对值，保留面积的正负号

def calculate_angle(vec1, vec2):
    """
    计算从vec1逆时针旋转到vec2的角度
    返回值在[0, 2π)范围内
    """
    angle = math.atan2(vec2, vec2) - math.atan2(vec1, vec1)
    if angle < 0:
        angle += 2 * math.pi
    return angle

def normalize_polygon(nodes):
    """将多边形标准化：从最小点开始"""
    if len(nodes) < 2:
        return nodes
    
    # 找到最小点（先比较x，再比较y）
    min_idx = 0
    for i in range(len(nodes) - 1):  # 不包括最后一个点（它等于第一个点）
        if (nodes[i].x < nodes[min_idx].x or 
            (nodes[i].x == nodes[min_idx].x and nodes[i].y < nodes[min_idx].y)):
            min_idx = i
    
    # 从最小点开始重新排列
    normalized = nodes[min_idx:len(nodes)-1] + nodes[:min_idx] + [nodes[min_idx]]
    return normalized

def polygons_equal(nodes1, nodes2):
    """判断两个多边形是否相同"""
    if len(nodes1) != len(nodes2):
        return False
    
    # 标准化
    norm1 = normalize_polygon(nodes1)
    norm2 = normalize_polygon(nodes2)
    
    # 比较
    return all(norm1[i] == norm2[i] for i in range(len(norm1)))

def generate_all_polygons_both_directions(edges):
    """使用左转和右转算法生成所有多边形"""
    
    all_polygons = []
    
    # 对每条边，尝试左转（最小逆时针角）和右转（最小顺时针角）
    for start_edge in edges:
        for turn_direction in ['left', 'right']:
            polygon_edges = []
            polygon_nodes = [start_edge.start]
            current_edge = start_edge
            visited_in_path = set()
            
            while True:
                # 使用id来追踪访问过的边，而不是Edge对象本身
                edge_id = id(current_edge)
                if edge_id in visited_in_path:
                    break
                
                polygon_edges.append(current_edge)
                polygon_nodes.append(current_edge.end)
                visited_in_path.add(edge_id)
                
                # 找到从current_edge.end出发的所有边
                outgoing = []
                for e in edges:
                    # 比较Node对象而不是引用
                    if (e.start.x == current_edge.end.x and 
                        e.start.y == current_edge.end.y):
                        outgoing.append(e)
                
                if len(outgoing) == 0:
                    break
                
                # 当前边的方向向量（确保是Node对象）
                vec1 = (current_edge.end.x - current_edge.start.x, 
                       current_edge.end.y - current_edge.start.y)
                
                # 找到下一条边
                best_edge = None
                
                if turn_direction == 'left':
                    # 左转：最小逆时针角
                    min_angle = float('inf')
                    for next_edge in outgoing:
                        # 确保next_edge.end是Node对象
                        vec2 = (next_edge.end.x - next_edge.start.x,
                               next_edge.end.y - next_edge.start.y)
                        angle = calculate_angle(vec1, vec2)
                        if angle < min_angle:
                            min_angle = angle
                            best_edge = next_edge
                else:
                    # 右转：最大逆时针角（即最小顺时针角）
                    max_angle = -float('inf')
                    for next_edge in outgoing:
                        vec2 = (next_edge.end.x - next_edge.start.x,
                               next_edge.end.y - next_edge.start.y)
                        angle = calculate_angle(vec1, vec2)
                        if angle > max_angle:
                            max_angle = angle
                            best_edge = next_edge
                
                if best_edge is None:
                    break
                
                # 检查是否回到起点（比较Node对象的坐标）
                if (best_edge.start.x == start_edge.start.x and 
                    best_edge.start.y == start_edge.start.y and
                    best_edge.end.x == start_edge.end.x and 
                    best_edge.end.y == start_edge.end.y):
                    break
                
                current_edge = best_edge
            
            # 检查是否形成闭合多边形
            if (len(polygon_edges) >= 3 and 
                polygon_nodes.x == polygon_nodes[-1].x and
                polygon_nodes.y == polygon_nodes[-1].y):
                area = polygon_area(polygon_nodes)
                all_polygons.append({
                    'edges': polygon_edges,
                    'nodes': polygon_nodes,
                    'area': area,
                    'direction': turn_direction
                })
    
    # 去重
    unique = []
    for poly in all_polygons:
        is_dup = False
        for u in unique:
            if polygons_equal(poly['nodes'], u['nodes']):
                is_dup = True
                break
        if not is_dup:
            unique.append(poly)
    
    return unique

# 执行多边形生成
all_polygons = generate_all_polygons_both_directions(edges)
print(f"找到 {len(all_polygons)} 个唯一多边形") math

# 判断多边形的方向，返回面积
def polygon_area(polygon):
    """
    计算多边形的有向面积
    输入polygon为多边形的结点坐标列表（二维元组），首尾结点相同
    返回值>0表示逆时针，<0表示顺时针
    """
    if not polygon:
        return 0.0
    # 确保闭合
    if polygon[0] != polygon[-1]:
        polygon = list(polygon) + [polygon[0]]
    area = 0.0
    for i in range(len(polygon) - 1):  # 这里用 n - 1，因为第一个点和最后一个点相同
        x1, y1 = polygon[i]
        x2, y2 = polygon[i + 1]
        area += x1 * y2
        area -= y1 * x2
    return area / 2.0  # 不取绝对值，保留面积的正负号

def calculate_angle(vec1, vec2):
    """
    计算从vec1逆时针旋转到vec2的角度
    返回值在[0, 2π)范围内
    """
    a1 = math.atan2(vec1[1], vec1[0])
    a2 = math.atan2(vec2[1], vec2[0])
    angle = a2 - a1
    if angle < 0:
        angle += 2 * math.pi
    return angle

def normalize_polygon(nodes):
    """将多边形标准化：从最小点开始（不改变方向）"""
    if len(nodes) < 2:
        return nodes
    
    pts = list(nodes)
    # 移除末尾可能存在的重复起点
    if pts[0] == pts[-1]:
        pts = pts[:-1]
    
    # 找到最小点（先比较x，再比较y）
    min_idx = 0
    for i in range(len(pts)):  # 不包括最后一个点（已去掉闭合点）
        if (pts[i][0] < pts[min_idx][0] or 
            (pts[i][0] == pts[min_idx][0] and pts[i][1] < pts[min_idx][1])):
            min_idx = i
    
    # 从最小点开始重新排列
    normalized = pts[min_idx:] + pts[:min_idx]
    # 重新闭合
    normalized.append(normalized[0])
    return normalized

def polygons_equal(nodes1, nodes2):
    """判断两个多边形是否相同（忽略起点与方向差异）"""
    if not nodes1 or not nodes2:
        return False
    
    # 去掉闭合点
    a = nodes1[:-1] if nodes1[0] == nodes1[-1] else nodes1
    b = nodes2[:-1] if nodes2[0] == nodes2[-1] else nodes2
    if len(a) != len(b):
        return False
    
    # 标准化（正向）
    na = normalize_polygon(a)
    nb = normalize_polygon(b)
    if na == nb:
        return True
    
    # 反向比较
    nb_rev = normalize_polygon(list(reversed(b)))
    return na == nb_rev

def to_directed_segments(edges_input):
    """
    将输入的edges变量（可能是多段折线或边列表）转换为有向边集合。
    每条边用((x1, y1), (x2, y2))表示。
    """
    base = []
    if isinstance(edges_input, (list, tuple)):
        for item in edges_input:
            # 形如[(x,y), (x,y), ...]的折线
            if isinstance(item, (list, tuple)) and len(item) >= 2 and isinstance(item[0], (list, tuple)):
                pts = []
                for p in item:
                    if isinstance(p, (list, tuple)) and len(p) == 2:
                        pts.append((float(p[0]), float(p[1])))
                for i in range(len(pts) - 1):
                    if pts[i] != pts[i + 1]:
                        base.append((pts[i], pts[i + 1]))
            # 形如((x1,y1),(x2,y2))的单条边
            elif isinstance(item, (list, tuple)) and len(item) == 2 and \
                 isinstance(item[0], (list, tuple)) and isinstance(item[1], (list, tuple)) and \
                 len(item[0]) == 2 and len(item[1]) == 2:
                p1 = (float(item[0][0]), float(item[0][1]))
                p2 = (float(item[1][0]), float(item[1][1]))
                if p1 != p2:
                    base.append((p1, p2))
    # 去重
    base = list({(a, b) for a, b in base})
    # 生成双向有向边
    directed = set()
    for a, b in base:
        directed.add((a, b))
        directed.add((b, a))
    return list(directed)

def generate_all_polygons_both_directions(edges_input):
    """使用左转和右转算法生成所有多边形（基于二维点元组）"""
    directed_edges = to_directed_segments(edges_input)
    if not directed_edges:
        return []
    
    # 出边映射
    outgoing_map = {}
    for a, b in directed_edges:
        outgoing_map.setdefault(a, []).append((a, b))
    
    all_polygons = []
    seen_polygons = []  # 存放已发现多边形的节点序列（用于去重）
    
    # 对每条有向边，尝试左转和右转
    for start_edge in directed_edges:
        for turn_direction in ['left', 'right']:
            polygon_nodes = [start_edge[0], start_edge[1]]
            current_edge = start_edge
            visited_dir_edges = set([current_edge])
            
            while True:
                curr_vec = (current_edge[1][0] - current_edge[0][0],
                            current_edge[1][1] - current_edge[0][1])
                
                outgoing = outgoing_map.get(current_edge[1], [])
                if not outgoing:
                    break
                
                best_edge = None
                if turn_direction == 'left':
                    # 左转：最小逆时针角
                    best_angle = float('inf')
                    for next_edge in outgoing:
                        next_vec = (next_edge[1][0] - next_edge[0][0],
                                    next_edge[1][1] - next_edge[0][1])
                        angle = calculate_angle(curr_vec, next_vec)
                        if angle < best_angle - 1e-12:
                            best_angle = angle
                            best_edge = next_edge
                else:
                    # 右转：最大逆时针角（即最小顺时针角）
                    best_angle = -float('inf')
                    for next_edge in outgoing:
                        next_vec = (next_edge[1][0] - next_edge[0][0],
                                    next_edge[1][1] - next_edge[0][1])
                        angle = calculate_angle(curr_vec, next_vec)
                        if angle > best_angle + 1e-12:
                            best_angle = angle
                            best_edge = next_edge
                
                if best_edge is None:
                    break
                
                # 检查是否回到起始有向边
                if best_edge == start_edge:
                    polygon_nodes.append(start_edge[0])  # 闭合
                    break
                
                # 防止在同一条路径中重复边导致死循环
                if best_edge in visited_dir_edges:
                    break
                
                visited_dir_edges.add(best_edge)
                polygon_nodes.append(best_edge[1])
                current_edge = best_edge
            
            # 检查是否形成闭合多边形
            if len(polygon_nodes) >= 4 and polygon_nodes[0] == polygon_nodes[-1]:
                area = polygon_area(polygon_nodes)
                if abs(area) > 1e-12:
                    # 去重
                    is_dup = False
                    for u in seen_polygons:
                        if polygons_equal(polygon_nodes, u):
                            is_dup = True
                            break
                    if not is_dup:
                        seen_polygons.append(polygon_nodes)
                        all_polygons.append({
                            'edges': None,   # 这里不再依赖自定义 Edge 类型
                            'nodes': polygon_nodes,
                            'area': area,
                            'direction': turn_direction
                        })
    
    return all_polygons

# 执行多边形生成
all_polygons = generate_all_polygons_both_directions(edges)
print(f"找到 {len(all_polygons)} 个唯一多边形")

AttributeError: 'list' object has no attribute 'x'

In [ ]:
# 按照从外到内的顺序排序（按面积绝对值降序）
sorted_polygons = sorted(all_polygons, key=lambda p: abs(p['area']), reverse=True)

# 重新编号
final_polygons = []
for i, poly in enumerate(sorted_polygons):
    poly['id'] = i + 1
    final_polygons.append(poly)

# 显示多边形信息
print("多边形列表：\n")
for poly in final_polygons:
    node_coords = [f"({n.x}, {n.y})" for n in poly['nodes']]
    print(f"Polygon {poly['id']}:")
    print(f"  结点序列: {' → '.join(node_coords)}")
    print(f"  面积: {poly['area']}")
    print(f"  方向: {'逆时针(CCW)' if poly['area'] > 0 else '顺时针(CW)'}")
    print()

## 关系表输出

In [ ]:
# 边-结点关系表
edge_node_data = []
for i, edge in enumerate(edges):
    edge_node_data.append({
        '边ID': i,
        '起点': f'({edge.start.x}, {edge.start.y})',
        '终点': f'({edge.end.x}, {edge.end.y})'
    })

edge_node_df = pd.DataFrame(edge_node_data)
print("边-结点关系表:")
print(edge_node_df.to_string(index=False))
print()

# 构建点-边关系
node_edge_data = []
for node in nodes:
    out_edges = []
    in_edges = []
    
    for i, e in enumerate(edges):
        if e.start.x == node.x and e.start.y == node.y:
            out_edges.append(i)
        if e.end.x == node.x and e.end.y == node.y:
            in_edges.append(i)
    
    if out_edges or in_edges:
        node_edge_data.append({
            '结点': f'({node.x}, {node.y})',
            '出边': ', '.join(map(str, out_edges)) if out_edges else '无',
            '入边': ', '.join(map(str, in_edges)) if in_edges else '无'
        })

node_edge_df = pd.DataFrame(node_edge_data)
print("结点-边关系表:")
print(node_edge_df.to_string(index=False))
print()

# 多边形-边关系表
polygon_edge_data = []

for poly in final_polygons:
    edge_ids = []
    for edge in poly['edges']:
        # 找到这条边在原始边列表中的索引
        for i, e in enumerate(edges):
            if (e.start.x == edge.start.x and e.start.y == edge.start.y and
                e.end.x == edge.end.x and e.end.y == edge.end.y):
                edge_ids.append(i)
                break
    
    polygon_edge_data.append({
        '多边形ID': poly['id'],
        '边序列': ', '.join(map(str, edge_ids)),
        '边数': len(edge_ids),
        '面积': poly['area']
    })

polygon_edge_df = pd.DataFrame(polygon_edge_data)
print("多边形-边关系表:")
print(polygon_edge_df.to_string(index=False))
print()

## matplotlib输出

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# 定义颜色
colors = ['blue', 'orange', 'green', 'red']

# 绘制每个多边形
for i, poly in enumerate(final_polygons):
    nodes_list = poly['nodes']
    x_coords = [n.x for n in nodes_list]
    y_coords = [n.y for n in nodes_list]
    
    # 绘制多边形边界
    ax.plot(x_coords, y_coords, color=colors[i], linewidth=2, label=f'Polygon {poly["id"]}')
    
    # 标注顶点坐标
    for j in range(len(nodes_list) - 1):  # 不标注最后一个点（它与第一个点重复）
        ax.annotate(f'({nodes_list[j].x}, {nodes_list[j].y})', 
                   xy=(nodes_list[j].x, nodes_list[j].y),
                   xytext=(5, 5), 
                   textcoords='offset points',
                   fontsize=8,
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.3))

# 设置坐标轴
ax.set_xlim(-0.5, 3.5)
ax.set_ylim(-0.5, 3.5)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.legend(loc='upper left', fontsize=10)
ax.set_xlabel('X', fontsize=12)
ax.set_ylabel('Y', fontsize=12)
ax.set_title('空间拓扑关系运算和生成 - 多边形可视化', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('polygons_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

## 简答题

> 如何在生成多边形的同时，记录弧段的左右多边形信息？

通过结合左转和右转算法实现弧段左右多边形的记录。

### 核心思路

左转算法（逆时针遍历）生成的多边形位于弧段的左侧，右转算法（顺时针遍历）生成的多边形位于弧段的右侧。

### 实现步骤
1. 初始化：为每条边创建 `left_polygon` 和 `right_polygon` 属性，初始值为 `None`
2. 左转遍历：从每条边出发，使用左转算法（选择逆时针方向角度最小的边）生成多边形。生成多边形后，将该多边形ID记录到路径中所有边的 `left_polygon` 属性
3. 右转遍历：从每条边出发，使用右转算法（选择顺时针方向角度最小的边）生成多边形。生成多边形后，将该多边形ID记录到路径中所有边的 `right_polygon` 属性
4. 去重处理：多个起始边可能生成相同的多边形，需要通过标准化节点序列进行去重
